# 셀프 주유소는 정말 저렴할까?

- 오피넷 검색

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## 서울시 구별 주유소 가격 정보 얻기

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.opinet.co.kr/user/main/mainView.do')

In [ ]:
# 싼 주유소 찾기 화면
driver.find_element_by_css_selector('.ic_m1').click()

In [ ]:
# 서울 자치구 리스트 가져오기
region = driver.find_element_by_xpath('//*[@id="SIGUNGU_NM0"]')
gu_list = region.find_elements_by_css_selector('option')
gu_names = [gu.text for gu in gu_list]
del gu_names[0]

In [ ]:
gu = gu_names[0]
region = driver.find_element_by_css_selector('#SIGUNGU_NM0')
region.send_keys(gu)

In [ ]:
# 엑셀 저장
driver.find_element_by_xpath('//*[@id="glopopd_excel"]').click()

In [ ]:
# 25개 자치구에 대해서 엑셀 저장
for gu in gu_names:
    region = driver.find_element_by_css_selector('#SIGUNGU_NM0')
    region.send_keys(gu)
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="glopopd_excel"]').click()
    time.sleep(2)

In [ ]:
driver.close()

## 구별 휘발유 가격에 대한 데이터 정리

In [ ]:
import pandas as pd 
from glob import glob

In [ ]:
glob('/Users/CPB06GameN/Downloads/지역*.xls')

In [ ]:
file = glob('/Users/CPB06GameN/Downloads/지역*.xls')[0]
df = pd.read_excel(file, header=2)
df.head()

In [ ]:
# 25개 파일을 읽어 하나의 데이터프레임으로 만듦
raw = []
for file in glob('/Users/CPB06GameN/Downloads/지역*.xls'):
    df = pd.read_excel(file, header=2)
    raw.append(df)

station_raw = pd.concat(raw)

In [ ]:
station_raw.info()

In [ ]:
stations = pd.DataFrame({
    '상호': station_raw.상호,
    '주소': station_raw.주소,
    '가격': station_raw.휘발유,
    '셀프': station_raw.셀프여부,
    '상표': station_raw.상표
})
stations.head()

In [ ]:
stations['구'] = [addr.split()[1] for addr in station_raw.주소]
stations.tail()

In [ ]:
stations['구'].unique()

In [ ]:
stations = stations[stations['가격'] != '-']
stations['가격'] = pd.to_numeric(stations['가격'])
stations.info()

## 셀프 주유소가 정말 저렴한지 확인

In [ ]:
import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mpl.rcParams['axes.unicode_minus'] = False 
plt.rcParams["font.family"] = 'Malgun Gothic'

In [ ]:
stations.boxplot(column='가격', by='셀프', figsize=(12,8));

In [ ]:
# 상표별로 확인
plt.figure(figsize=(12,8))
sns.boxplot(x='상표', y='가격', hue='셀프', data=stations)
plt.show()

## 서울시 구별 휘발유 가격 확인

In [ ]:
import json
import folium
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# 가격이 비싼 주유소 Top 5
stations.sort_values(by='가격', ascending=False).head(5)

In [ ]:
# 가격이 착한 주유소 Top 5
stations.sort_values(by='가격').head(5)

In [ ]:
gu_data = stations.groupby('구')[['가격']].mean()
gu_data

In [ ]:
geo_path = '../04.지도_공공API/skorea_municipalities_geo_simple.json'
geo_data = json.load(open(geo_path, encoding='utf-8'))